In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df=pd.read_csv("heart_clean_final.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,62,0,0,138,294.0,1,1,106,0,1.9,1,3,2,0
1,58,0,0,100,248.0,0,0,122,0,1.0,1,0,2,1
2,58,1,0,114,318.0,0,2,140,0,3.5,0,3,1,0
3,54,1,0,122,286.0,0,0,116,1,3.2,1,2,2,0
4,71,0,0,112,149.0,0,1,125,0,1.6,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,0,2,120,178.0,1,1,100,0,0.0,2,0,2,1
179,64,1,2,140,335.0,0,1,158,0,0.0,2,0,2,0
180,68,0,2,120,211.0,0,0,115,0,1.5,1,0,2,1
181,44,0,2,108,141.0,0,1,175,0,0.6,1,0,2,1


In [5]:
indep=df.drop("target",axis=1)
dep=df["target"]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from lightgbm import LGBMClassifier

In [35]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators':[50,100,200],'max_depth':[5,10,15],'learning_rate':[0.01,0.05,0.1],'subsample':[0.7,0.8,1.0],'colsample_bytree':[0.7,0.8,1.0]} 
grid = GridSearchCV(LGBMClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[LightGBM] [Info] Number of positive: 85, number of negative: 37
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 152
[LightGBM] [Info] Number of data points in the train set: 122, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.696721 -> initscore=0.831733
[LightGBM] [Info] Start training from score 0.831733
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

GridSearchCV(estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 1.0],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [5, 10, 15],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.7, 0.8, 1.0]},
             scoring='f1_weighted', verbose=3)

In [37]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [39]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.7}: 0.8094697057527768


In [41]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[ 5  7]
 [ 4 45]]


In [43]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.56      0.42      0.48        12
           1       0.87      0.92      0.89        49

    accuracy                           0.82        61
   macro avg       0.71      0.67      0.68        61
weighted avg       0.80      0.82      0.81        61



In [45]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8265306122448979

In [47]:
table=pd.DataFrame.from_dict(re)

In [49]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.070880,0.013008,0.003405,0.000377,0.7,0.01,5,50,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,217
1,0.073343,0.011558,0.004704,0.000873,0.7,0.01,5,50,0.8,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,217
2,0.060353,0.006745,0.003860,0.000804,0.7,0.01,5,50,1.0,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.550476,0.550476,0.587398,0.587398,0.587398,0.572630,0.018088,217
3,0.088864,0.008674,0.003404,0.000375,0.7,0.01,5,100,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.767366,0.871019,0.747151,0.736111,0.747151,0.773760,0.049666,199
4,0.093473,0.002877,0.003505,0.000549,0.7,0.01,5,100,0.8,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.767366,0.871019,0.747151,0.736111,0.747151,0.773760,0.049666,199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.077022,0.004351,0.003201,0.000399,1.0,0.10,15,100,0.8,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",0.833016,0.802852,0.824074,0.736111,0.871978,0.813606,0.044754,154
239,0.085124,0.008753,0.004109,0.000864,1.0,0.10,15,100,1.0,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",0.833016,0.802852,0.824074,0.736111,0.871978,0.813606,0.044754,154
240,0.156347,0.007924,0.003603,0.000373,1.0,0.10,15,200,0.7,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",0.785031,0.760000,0.786630,0.736111,0.795455,0.772645,0.021743,208
241,0.146237,0.015619,0.003805,0.000683,1.0,0.10,15,200,0.8,"{'colsample_bytree': 1.0, 'learning_rate': 0.1...",0.785031,0.760000,0.786630,0.736111,0.795455,0.772645,0.021743,208
